# Prepare driver

In [1]:
! wget -P /home/jovyan https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar

--2022-07-20 14:20:35--  https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.196.209, 199.232.192.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.196.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 825943 (807K) [application/java-archive]
Saving to: ‘/home/jovyan/postgresql-42.2.5.jar’

postgresql-42.2.5.j 100%[===================>] 806.58K  3.22MB/s    in 0.2s    

2022-07-20 14:20:36 (3.22 MB/s) - ‘/home/jovyan/postgresql-42.2.5.jar’ saved [825943/825943]



# Start spark application

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /home/jovyan/postgresql-42.2.5.jar --jars /home/jovyan/postgresql-42.2.5.jar pyspark-shell'

import pyspark

spark = pyspark.sql.SparkSession.builder \
        .master("local[1]") \
        .appName("snapshot") \
        .getOrCreate()

print("Application started")

Application started


# Warm-up spark

In [3]:
spark.sparkContext.range(1000).sum()
print("Spark application is ready for work")

Spark application is ready for work


# Read some PostgreSQL Data

In [4]:
customers = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='customers'
    ).load()

products = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='products'
    ).load()

orders = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='orders'
    ).load()

customers.registerTempTable("customers")
products.registerTempTable("products")
orders.registerTempTable("orders")

print("Customers table")
customers.show(5)
print("Orders table")
orders.show(5)
print("Products table")
products.show(5)


/usr/local/spark/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Customers table
+----+----------+---------+--------------------+
|  id|first_name|last_name|               email|
+----+----------+---------+--------------------+
|1001|     Sally|   Thomas|sally.thomas@acme...|
|1002|    George|   Bailey|  gbailey@foobar.com|
|1003|    Edward|   Walker|       ed@walker.com|
|1004|      Anne|Kretchmar|  annek@noanswer.org|
+----+----------+---------+--------------------+

Orders table
+-----+----------+---------+--------+----------+
|   id|order_date|purchaser|quantity|product_id|
+-----+----------+---------+--------+----------+
|10001|2016-01-16|     1001|       1|       102|
|10002|2016-01-17|     1002|       2|       105|
|10003|2016-02-19|     1002|       2|       106|
|10004|2016-02-21|     1003|       1|       107|
+-----+----------+---------+--------+----------+

Products table
+---+------------------+--------------------+------+
| id|              name|         description|weight|
+---+------------------+--------------------+------+
|101|      

# Read and join the data

In [26]:
# please write your query here
query = "select * from customers"
result = spark.sql(query)
(result
 .selectExpr("*", "CURRENT_TIMESTAMP() as Load_Dttm")
 .write
 .format("parquet")
 .mode("overwrite")
 .save("/home/jovyan/weight_report")
)

In [27]:
spark.read \
    .format("parquet").load("/home/jovyan/weight_report").show()

+----+----------+---------+--------------------+--------------------+
|  id|first_name|last_name|               email|           Load_Dttm|
+----+----------+---------+--------------------+--------------------+
|1001|     Sally|   Thomas|sally.thomas@acme...|2022-07-20 14:51:...|
|1002|    George|   Bailey|  gbailey@foobar.com|2022-07-20 14:51:...|
|1003|    Edward|   Walker|       ed@walker.com|2022-07-20 14:51:...|
|1004|      Anne|Kretchmar|  annek@noanswer.org|2022-07-20 14:51:...|
+----+----------+---------+--------------------+--------------------+



In [28]:
spark.stop()